In [ ]:
# Here is a little matrix class with some random stuff from linear algrebra
# I know I should use numpy but I wanted to implement it myself (even if it is less efficient)

In [318]:
# A few basic functions
def dot_product(v1,v2):
    total = 0
    for x, y in zip(v1,v2):
        total += x*y
    return total

def add_vectors(v1,v2):
    return [x+y for x, y in zip(v1,v2)]

def subtract_vectors(v1,v2):
    return [x-y for x, y in zip(v1,v2)]

def scalar_multiply(c, v):
    return [c*x for x in v]

def vector_length(v):
    return sqrt(sum([(a**2) for a in v]))

In [319]:
from fractions import Fraction

# Gram-Schmidt: from a basis create find an orthogonal basis
def gram_schmidt(basis, fraction=True):
    # v1 = x1 always
    basis = [[Fraction(x) for x in v] for v in basis]
    orthogonal_basis = [basis[0]]

    # vp = xp - (dot(xp,vp-1)/dot(vp-1,vp-1))(vp-1) -...- (dot(xp,v1)/dot(v1,v1))(v1)
    for vector in basis[1:]:
        for ortho_vector in orthogonal_basis:
            temp = scalar_multiply((dot_product(vector, ortho_vector) /  dot_product(ortho_vector, ortho_vector)), 
                                   ortho_vector)
            
            vector = subtract_vectors(vector, temp)
        orthogonal_basis.append(vector)
    
    return orthogonal_basis

def orthonormal_basis(basis, fraction=True):
    orthogonal_basis = gram_schmidt(basis, fraction=fraction)
    
    columns = []
    for column in range(0,len(self.matrix[0])):
        columns.append([row[column] for row in basis])
    
    orthonormal_basis = []
    for column in columns:
        length = vector_length(column)
        orthonormal_basis.append([(x/vector_length) for x in column])
    
    rows = []
    for row in range(0,len(self.matrix[0])):
        rows.append([column[row] for column in orthonormal_basis])
        
    return rows

In [10]:
class Matrix:
    """Matrix class for random things you can do to a matrix, may add bases to this
    because you can essentially treat them as matrices"""
    
    def __init__(self, matrix):
        "Create mxn matrix"
        try:
            n = len(matrix[0])
        except TypeError:
            raise Exception("Matrix should be constructed from lists")
        
        for column in matrix:
            if len(column) != n:
                raise Exception("Rows must be the same length")
        
        self.matrix = matrix
       
    
    def size(self):
        return (len(self.matrix), len(self.matrix[0]))
    
    def isSquare(self):
        "Check if the matrix is square"
        return len(self.matrix) == len(self.matrix[0])
    
    
    def __gcd(self,a,b):
        return a if (b==0) else gcd(b,a%b)
    
    def transpose(self):
        columns = []
        for column in range(0,len(self.matrix[0])):
            columns.append([row[column] for row in self.matrix])
        return columns
    
    def columnsAreScalarMultiples(self):
        A = self.transpose()
        for col_ix in range(0,len(A)-1):
            temp = []
            for row_ix in range(0,len(A[0])):
                if (A[col_ix][row_ix] == 0 and A[col_ix+1][row_ix] != 0) \
                or (A[col_ix+1][row_ix] == 0 and A[col_ix][row_ix] != 0):
                    return False
                temp.append(A[col_ix][row_ix]/A[col_ix+1][row_ix])
            for ix in range(0,len(temp)-1):
                if temp[ix] != temp[ix+1]:
                    return False
        return True
            
    
    def echelonForm(self, matrix="self"):
        "Use row reduction to get echelon form of matrix"
        A = self.matrix if matrix == "self" else matrix
        row = 0
        column = 0

        while row < len(A) and column < len(A[0]):
            if A[row][column] != 0:
                A[row] = [x/A[row][column] for x in A[row]]

                for i in range(row+1,len(A)):
                        if A[i][column] != 0:
                            A[i] = [x + y for x, y in zip(A[i], [x*-A[i][column] for x in A[column]])]
            column+=1
            row+=1
            
        # find pivot with 1 in first position TODO?
        return A
    
    def reducedEchelonForm(self, matrix="self"):
        A = self.echelonForm()
        
        for r_ix, row in enumerate(reversed(A)):
            try:
                pivot_column = row.index(1)
                pivot_row = len(A) - (r_ix+1)
                # inner loop
                for ri, r in enumerate(A):
                    if r[pivot_column] != 0 and ri != pivot_row:
                        A[ri] = [x + y for x, y in zip(A[ri], [x*-A[ri][pivot_column] for x in A[pivot_row]])]
            except:
                pass
        return A
    
    def __reducedEchelonForm(self,A):
        for r_ix, row in enumerate(reversed(A)):
            try:
                pivot_column = row.index(1)
                pivot_row = len(A) - (r_ix+1)
                # inner loop
                for ri, r in enumerate(A):
                    if r[pivot_column] != 0 and ri != pivot_row:
                        A[ri] = [x + y for x, y in zip(A[ri], [x*-A[ri][pivot_column] for x in A[pivot_row]])]
            except:
                pass
        return A
    
    
    def __determinant_helper(self, matrix):
        if len(matrix) == 2:
            return (matrix[0][0]*matrix[1][1]) - (matrix[1][0]*matrix[0][1])

        # alternatively, find row or column with fewest non-zero values TODO

        total = 0

        # make the minor
        for ix, value in enumerate(matrix[0]):
            if value != 0:
                temp_mat = []
                for i, row in enumerate(matrix):
                    if i != 0:
                        temp_row = []
                        for j, val in enumerate(row):
                            if j != ix:
                                temp_row.append(val)
                        temp_mat.append(temp_row)

                # -1^(j+1)a1jdetA1j
                total += ((-1)**(ix)) * value * determinant(temp_mat)
        return total
    
    def determinant(self):
        if self.isSquare():
            return self.__determinant_helper(self.matrix)
    
    # fix memory stuff...
    def inverse(self):
        # all the invertible checks go here...
        if not self.isSquare():
            raise Exception("Matrix must be square to be invertible")
        
        identity_matrix = []
        for x in range(0,len(self.matrix)):
            temp = []
            for y in range(0,len(self.matrix)):
                if x == y:
                    temp.append(1)
                else:
                    temp.append(0)
            identity_matrix.append(temp)
                    
        A = self.matrix
        for ix, (a_row, i_row) in enumerate(zip(A,identity_matrix)):
            A[ix] = a_row + i_row
        
        # got a little frustrated with some functions. just made a new matrix instead.
        temp = Matrix(A)
        return temp.reducedEchelonForm() # get the matrix that was appended and multiply everything by the determinant

In [ ]:
# feel free to ignore this stuff

In [317]:
M = Matrix([[1,2],[5,10]])
M.columnsAreScalarMultiples()

True

In [291]:
M = Matrix([[1, 2, 1, 0],[0, -2, -3, 1]])

In [305]:
M = Matrix([[1,0,3],[-1,1,-6],[2,4,1]])
M.echelonForm()

[[1.0, 0.0, 3.0], [0.0, 1.0, -3.0], [0.0, 0.0, 1.0]]

In [307]:
M.getColumns()

[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [3.0, -3.0, 1.0]]

In [220]:
M.reducedEchelonForm()

[[1.0, 0.0, -2.0, 1.0], [-0.0, 1.0, 1.5, -0.5]]

In [139]:
for r_ix, row in enumerate(reversed(M.echelonForm())):
    try:
        pivot_column = row.index(1)
        pivot_row = len(A) - (r_ix+1)
        for x in A:
            print(x)
        # inner loop
        for ri, r in enumerate(A):
            if r[pivot_column] != 0 and ri != pivot_row:
                A[ri] = [x + y for x, y in zip(A[ri], [x*-A[pivot_row][pivot_column] for x in A[pivot_row]])]
    except:
        pass

0 0
1 1
2 2
[1.0, 1.0, -0.3333333333333333]
[-0.0, 1.0, -0.0]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[1.0, 1.0, -0.3333333333333333] hey
[1.0, 0.0, -0.3333333333333333]
[-0.0, 1.0, -0.0]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[[1.0, 0.0, -0.3333333333333333], [-0.0, 1.0, -0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]


In [132]:
M.echelonForm()

0 0
1 1
2 2


[[1.0, 2.0, -0.3333333333333333],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0]]

In [111]:
A

[[1.0, 2.0, -0.3333333333333333],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0]]

In [176]:
x = Matrix([[-.7,.3,.3],[.4,-.9,.5]])

In [177]:
x.isSquare()

False

In [178]:
x.reducedEchelonForm()

[[1.0, 0.0, -0.8235294117647061], [-0.0, 1.0, -0.9215686274509806]]

In [284]:
# determinant found using cofactor expansion
E = [[1,0,3],
     [-1,1,-6],
     [2,4,1]]

def determinant(matrix):
    if len(matrix) == 2:
        return (matrix[0][0]*matrix[1][1]) - (matrix[1][0]*matrix[0][1])
    
    # alternatively, find row or column with fewest non-zero values TODO
    
    total = 0
    
    # make the minor
    for ix, value in enumerate(matrix[0]):
        if value != 0:
            temp_mat = []
            for i, row in enumerate(matrix):
                if i != 0:
                    temp_row = []
                    for j, val in enumerate(row):
                        if j != ix:
                            temp_row.append(val)
                    temp_mat.append(temp_row)
            
            # -1^(j+1)a1jdetA1j
            total += ((-1)**(ix)) * value * determinant(temp_mat)
    return total

In [285]:
determinant(E)

1
-6
4
1
[[1, -6], [4, 1]]
-1
1
2
4
[[-1, 1], [2, 4]]


7